In [62]:
pip install python-dotenv 'crewai[tools]' langchain-openai -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [63]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [64]:
from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [65]:
llm = ChatOpenAI(model="gpt-5.2", temperature=0.5)

In [66]:
interviewer = input("Enter the name of the interviewer: ")
# Adrian Hajdin
company = input("Enter the name of the company: ")
# JS Mastery
job_position = input("Enter the job position: ")
# Desenvolvedor(a) Full Stack Júnior
job_description = input("Enter the job description: ")
# Apoiar o desenvolvimento de novas funcionalidades em aplicações web (frontend e backend), sob orientação de desenvolvedores(as) mais experientes
# Participar de atividades de manutenção, correção de bugs e melhorias contínuas
# Colaborar na revisão de código e aprender boas práticas de engenharia (testes, versionamento, arquitetura)
# Apoiar integrações com APIs internas e externas
# Contribuir para a documentação técnica dos sistemas
# Participar de cerimônias do time (dailies, refinamentos, retrospectivas) com postura ativa e colaborativa
# Demonstrar evolução contínua em autonomia, domínio técnico e capacidade de entrega

# Research Agent

In [78]:
backstory = """Como um Agente de pesquisa, sua missão é descobrir detalhes e informações relevantes.
                O objetivo final é cumprir expectativas da atividade"""
research_agent = Agent(
    role="Agente de pesquisa",
    goal="Conduzir pesquisa aprofundada",
    backstory=backstory,
    llm=llm,
    tools=[ScrapeWebsiteTool(), SerperDevTool()],
)

In [68]:
research_company_task = Task(
    description=f"Pesquise sobre {company}, a posição deles no setor e as trends recentes da industria. Foque em noticias recentes e desenvolvimentos nos ultimos 6 meses",
    expected_output=f"Um report compreensivo em Markdown com bullet points sobre {company}",
    agent=research_agent,
)

In [69]:
research_person_task = Task(
    description=f"Pesquisar sobre {interviewer} com foco na sua carreira profissional",
    expected_output=f"Um report compreensivo em Markdown com bullet points sobre {interviewer}",
    agent=research_agent,
)

# Interview agent

In [70]:
interviewer_agent = Agent(
    role=f"Você é {interviewer}, empregado na {company}, e está contratando para a vaga {job_position}",
    goal=f"Você ajuda o usuário a se preparar para a entrevista de emprego para a vaga {job_position} com a seguinte descrição: {job_description}",
    backstory="Você é um entrevistador de uma empresa de tecnologia",
    llm=llm,
)


In [71]:
interview_prep_task = Task(
    description=f"Entreviste o usuário para a vaga {job_description} com as seguintes descrições: {job_description}",
    expected_output=f"""
Pergunte apenas uma pergunta para o usuário que seja relevante a vaga {job_position}
""",
    agent=interviewer_agent,
    human_input=True,
)

# AI Coach

In [72]:
coach_agent = Agent(
    role="Coach de entrevista",
    goal=f"Eu ajudo o usuário a se preparar para a entrevista de emprego para a posição {job_position} avaliando a relevância da resposta.",
    backstory=f"Você é um expert em entrevista de empregos em empresas {company}",
    llm=llm,
)

In [73]:
description = f"Defina uma lista de questões baseada na empresa {company}, descrição de vaga {job_description} e entrevistador {interviewer}"

define_questions_task = Task(
    description=description,
    expected_output="""Uma lista de 20 perguntas no formato Markdown que cubra: Fit cultural e de time, Fit para vaga, Background e jeito de trabalhar, Mentalidade de crescimento""",
    agent=coach_agent,
    context=[research_company_task, research_person_task],
)

In [74]:
coaching_task = Task(
    description="Dê feedback para o usuário sobre sua resposta",
    expected_output="Dê feedback para o usuário sobre sua resposta com bullet points no que foi bom, ruim, e no que pode ser levado para o proximo nível",
    agent=coach_agent,
    context=[interview_prep_task],
)

# AI Crew

In [ ]:
crew = Crew(
    agents=[research_agent, coach_agent],
    tasks=[research_person_task, research_company_task, define_questions_task],
    verbose=True,
    process=Process.sequential,
)
result = crew.kickoff(
    {"topic": "Write a list of question to prepare for the interview"}
)


In [84]:
display(Markdown(result.tasks_output[2].raw))

## Lista de 20 perguntas (Eyna — Dev Full Stack Júnior | Entrevistador: Matheus)

### Fit cultural e de time
1. **Como você gosta de trabalhar no dia a dia com um time multidisciplinar (ex.: ML/Computer Vision + Backend + Front)?** Dê um exemplo real de colaboração.
2. **Conte uma situação em que você recebeu um feedback técnico direto (code review, PR, arquitetura).** O que você mudou depois disso?
3. **Quando você discorda de uma decisão técnica do time, como você aborda a conversa?** (ex.: dados, POC, trade-offs, alinhamento).
4. **Como você equilibra autonomia com alinhamento?** Ex.: quando você toca uma tarefa sozinho vs. quando você chama alguém cedo.
5. **Você já precisou “puxar” documentação/registro de conhecimento mesmo sem ninguém pedir?** Como decidiu o que documentar e qual formato usou?

### Fit para a vaga (APIs, integrações, edge+nuvem, AWS, observabilidade)
6. **Descreva uma API (REST ou GraphQL) que você já criou ou manteve.** Quais endpoints, autenticação, versionamento e principais decisões de design?
7. **Como você pensaria uma integração para clientes consumirem modelos de visão computacional “em escala”?** Quais seriam suas preocupações com latência, custo, filas e limites?
8. **Se um cliente precisa rodar inferência no edge e também na nuvem, como você desenharia o fluxo ponta a ponta?** O que fica no dispositivo vs. no backend?
9. **O que significa, na prática, uma solução ser “performática, segura, observável e fácil de manter”?** Cite exemplos concretos (métricas, logs, tracing, auth, testes, padrões).
10. **Como você trataria upload e processamento de imagens/vídeos em um backend?** (ex.: S3, pré-assinadas, validação, tamanho, vírus, formatos, retries).
11. **Em AWS, quando você escolheria Lambda vs ECS (ou outra opção)?** Quais critérios (cold start, tempo de execução, custo, operação, escalabilidade)?
12. **Quais sinais você monitoraria no CloudWatch (ou stack equivalente) para uma API em produção?** Dê exemplos de alarmes e métricas.
13. **Como você lidaria com erros e idempotência em integrações?** Ex.: reprocessamento, duplicidade, “at least once”, chaves idempotentes.
14. **Explique como você estruturaria autenticação e autorização para APIs consumidas por clientes externos.** (ex.: API keys, JWT, OAuth, RBAC, rate limiting).

### Background e jeito de trabalhar (engenharia, qualidade, Git, entrega)
15. **Me conte sobre sua experiência anterior (estágio/projeto) mais parecida com essa vaga.** Qual foi seu impacto e o que você entregou de ponta a ponta?
16. **Como você organiza uma tarefa do início ao fim?** (entendimento do problema, quebra em subtarefas, definição de pronto, validação, PR).
17. **Qual seu nível de conforto com bancos de dados (SQL/NoSQL)?** Dê um exemplo de modelagem simples que você já fez e como garantiu consistência/performance.
18. **Descreva seu fluxo com Git e Pull Requests.** Como você escreve commits, abre PR, responde review e evita conflitos?
19. **Você já escreveu testes (unitários/integrados) em algum projeto?** O que você escolheu testar e como isso ajudou (ou não) na manutenção?

### Mentalidade de crescimento (aprendizado rápido, IA como alavanca, ensinar o time)
20. **Como você usa IA para aumentar produtividade sem perder qualidade?** Dê um exemplo real (gerar testes, documentação, debugging) e como você valida o resultado antes de subir para produção.

In [87]:
# export company report
file_name = f"{company}_report.txt"

with open(file_name, "w") as file:
    file.write(result.tasks_output[0].raw)

# export interviewer questions
file_name = f"{interviewer}_report.txt"

with open(file_name, "w") as file:
    file.write(result.tasks_output[1].raw)

# export interview questions
file_name = f"{interviewer}_report.txt"

with open(file_name, "w") as file:
    file.write(result.tasks_output[2].raw)
